In [185]:
import pandas as pd
import ipywidgets as wg
import math

## Calculo de Reforço Pavimento Flexivel
### (base granular espessura =< 45cm)

In [186]:
N_titulo = wg.HTML(value="1) Informar o Nº USACE >>>")

N = wg.FloatText(value=5.99e7)

wg.HBox([N_titulo,N])

In [187]:
layout03 = wg.Layout(width='60%', height='30px')
h_exist_Titulo = wg.HTML(value="2) Informar a espessura do pavimento existente (0 a 20) :     ")
h_exist = wg.IntSlider(value=0,min=0,max=20,step=1, layout = layout03)
wg.HBox([h_exist_Titulo,h_exist])

In [188]:
layout02 = wg.Layout(width='80%', height='80px')

deflexoes_Titulo = wg.HTML(value="3) informar as deflexões do pavimento por estaca, em numeros inteiros (sem virgula)")
deflexoes = wg.Textarea( value='26 28 27 30 31 33 14 ', layout=layout02)
wg.HBox([deflexoes_Titulo,deflexoes])

In [189]:
layout01 = wg.Layout(width='100%', height='50px', justify ='right')

bt_calcular = wg.Button(description='Calcular', disabled=False, button_style='success',layout=layout01)



wg.HBox([bt_calcular])

1
DADOS   Num Estacas               7.000000
        Espessura do Projeto      2.000000
        Média                    27.000000
        Desvio Padrão             6.220000
        Deflexão Caracteristica  33.220000
PRO-11  Deflexão Admissivel      43.769630
        Espessura Reforço        -4.790930
PRO-269 Deflexão Admissivel      48.511394
        Espessura Reforço       -10.346307

Output(layout=Layout(border='1px solid black'))

In [190]:
def reforco_Pro_269(Dc):
    
    hef = -5.737 + (807.961/Dc) + 0.972
    
    if hef < h_exist.value:
        hef = h_exist.value
    hef
    
    Dadm = math.pow(10,(3.148-0.188*math.log(N.value,10)))
    hr = -19.015+(238.14/math.sqrt(Dadm))-1.357*hef+1.016
    return Dadm,hr
    

In [191]:
def reforco_Pro_11(Dc):
    
    Dadm = math.pow(10,(3.01-0.176*math.log10(N.value)))
    h = 40*(math.log(Dc/Dadm,10))
    return Dadm,h

In [192]:
def fun_calcular2(a):
    array = deflexoes.value.split()
    array_int = map(int,array) # < converter string in integer
    Serie = pd.Series(array_int)
    Dc= round((Serie.mean()+Serie.std()),2)
    
    Dadm_269,hr_269= reforco_Pro_269(Dc)
    Dadm_11,hr_11= reforco_Pro_11(Dc)
    
    nivel00 = ['DADOS']*5+ ['PRO-11']*2 + ['PRO-269']*2
    nivel01 = ['Num Estacas','Espessura do Projeto','Média','Desvio Padrão','Deflexão Caracteristica',
                  'Deflexão Admissivel', 'Espessura Reforço',
                  'Deflexão Admissivel', 'Espessura Reforço'
              ]
    
    nivel_zip = list(zip(nivel00,nivel01))
    indice_mult = pd.MultiIndex.from_tuples(nivel_zip)
    
    serie_dados= [1,[Serie.size,h_exist.value, round(Serie.mean(),2),round(Serie.std(),2),Dc
                     ,Dadm_11,hr_11,
                     Dadm_269,hr_269]]
    
    df = pd.DataFrame(serie_dados[1], index=indice_mult, columns=[1])
    linha_divisa = wg.Output(layout={'border': '1px solid black'})
    
    display(df,linha_divisa)   
bt_calcular.on_click(fun_calcular2)    

In [193]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Esconder/Mostrar <Codigo>"></form>''')